In [10]:
# import libraries
import pymongo
import pandas as pd
import os
import json
import time

from IPython.display import display, HTML
# from ipywidgets import widgets


from pyspark.sql import SparkSession
import pandas as pd
import os 

In [11]:
# to connect from VM system
mongo_client = pymongo.MongoClient("mongodb://localhost:27017/")

In [12]:
#path from where we will read files for one hour 
path = '/home/bdm/data_sources/stock_data_parquet/'
appName = "stock parquet files"
spark = SparkSession.builder.appName(appName).getOrCreate()

/home/bdm/.local/lib/python3.6/site-packages/pyspark/context.py:238: FutureWarning: Python 3.6 support is deprecated in Spark 3.2.
  FutureWarning


In [15]:
# define Database
tweetdb = mongo_client["sense_stock_db"]['tweet_data']

In [21]:
# get todays date according to indian timezone
# keeping buffer time to give time for processing (2 minutes)
buffer_minutes = 2
IN_time = pd.Timestamp.now('Asia/Kolkata')-pd.Timedelta(minutes=2)

date = str(IN_time.date()).replace("-","_")
date = "2022_06_03"

hour = int(IN_time.hour)
hour = 21

minute = int(IN_time.minute)
minute = 56



# run query to extract result
myquery = { "date": date , "companies":company, "hour":hour, "minute":minute}
mydoc = tweetdb.find(myquery)
for x in mydoc:
    print("Extracted data")
    df = pd.DataFrame(x['top_N_tweets'])
    display(HTML(df.to_html()))

Extracted data


,polarity,timestamp,sentiment,text,support_count
0,0.0,2022-06-03 16:26:59.547,Neutral,@drbillgibson Over reliance on antibiotics in these cases. Just encouraging resistance. Topical oestrogen is the way to go.,0


In [22]:
# define company
company = "RELIANCE"

In [23]:
def get_time():
    # get todays date according to indian timezone
    IN_time = pd.Timestamp.now('Asia/Kolkata')-pd.Timedelta(minutes=10)

    date = str(IN_time.date()).replace("-","_")
    #date = "2022_06_03"

    hour = int(IN_time.hour)
    #hour = 21

    minute = int(IN_time.minute)

    return date,hour,minute

In [47]:
def display_tweets(date,hour,minute,company="RELIENCE"):
    # run query to extract result
    myquery = { "date": date , "companies":company, "hour":hour, "minute":minute}
    mydoc = tweetdb.find(myquery)
    for x in mydoc:
        print("Extracted data")
        df = pd.DataFrame(x['top_N_tweets'])
    display(HTML(df.to_html()))

In [48]:
def display_stocks(date,hour,minute,company="RELIENCE"):
    date_ = os.path.join(path,date)
    parquet_name = 'stock_data'+'_'+str(hour)+'_'+str(minute)+'.parquet'
    file_name = os.path.join(date_,parquet_name)
    print(file_name)
    df1 = spark.read.parquet(file_name)
    df2 = df1.select('symbol','lastPrice','dayHigh','dayLow','closePrice','open')
    pd_df = df2.toPandas()
    df_cmp = pd_df[pd_df['symbol']==company]
    print(df_cmp)
    # display(HTML(df_cmp.to_html()))

In [54]:
# display top tweets for entered company each minute
while(True):
    
    date,hour,minute = get_time()
    # comp = input("Enter company name")
    comp = "RELIENCE"
    
    print("Top tweets for date - {} hour {} and minute {} for company {}".format(date,hour,minute, comp))

    # run query to diplay tweets
    try:
        myquery = { "date": date , "companies":company, "hour":hour, "minute":minute}
        mydoc = tweetdb.find(myquery)
        for x in mydoc:
            df = pd.DataFrame(x['top_N_tweets'])
        display(HTML(df.to_html()))
    except:
        print("no tweet for this minute")
        
    
    
    # run query to diplay stocks
    date_ = os.path.join(path,date)
    parquet_name = 'stock_data'+'_'+str(hour)+'_'+str(minute)+'.parquet'
    file_name = os.path.join(date_,parquet_name)
    df1 = spark.read.parquet(file_name)
    df2 = df1.select('symbol','lastPrice','dayHigh','dayLow','closePrice','open')
    pd_df = df2.toPandas()
    df_cmp = pd_df[pd_df['symbol']==company]
    display(HTML(df_cmp.to_html()))
    
    # sleep for around 1 minute
    time.sleep(60)

2022_06_05 22 22 RELIENCE


,polarity,timestamp,sentiment,text,support_count
0,0.0,2022-06-05 16:52:51.513,Neutral,RT @narendramodi: 130 crore Indians have decided that they will make India Aatmanirbhar. Our push for self-reliance is driven by a vision o…,0
1,-0.5,2022-06-05 16:52:56.168,Negative,"RT @Ranal55: If you are not relying on God, you are relying on the wrong things.\n\n#quote #quotesmith55 #God #reliance #SundaySpirit https:/…",0


/home/bdm/data_sources/stock_data_parquet/2022_06_05/stock_data_22_22.parquet


,symbol,lastPrice,dayHigh,dayLow,closePrice,open
0,RELIANCE,2778.0,2817.35,2742.0,2779.5,2744.0


2022_06_05 22 23 RELIENCE


...

,symbol,lastPrice,dayHigh,dayLow,closePrice,open
0,RELIANCE,2778.0,2817.35,2742.0,2779.5,2744.0


2022_06_05 22 25 RELIENCE


,polarity,timestamp,sentiment,text,support_count
0,0.5,2022-06-05 16:55:30.087,Positive,RT @gods_txt: Patience is the trait of mothers. Love is the trait of fathers. Reliance is the trait of slaves. Authority is the trait of ki…,0
1,0.0,2022-06-05 16:55:09.473,Neutral,@ZestMoneyCares @ZestMoney Dear team i have purchased mobile phones from reliance digital. monthly emi is 12350 pe… https://t.co/jbnoQN4Q00,0
2,-0.38035715,2022-06-05 16:55:34.929,Negative,@RelianceDigital \nCity: Pilibangan ; District: Hanumangarh; Rajasthan\nI am very disappointed from reliance smart Pi… https://t.co/nvUMlCMrIy,0


/home/bdm/data_sources/stock_data_parquet/2022_06_05/stock_data_22_25.parquet


,symbol,lastPrice,dayHigh,dayLow,closePrice,open
0,RELIANCE,2778.0,2817.35,2742.0,2779.5,2744.0


In [50]:
# display top tweets for entered company each minute
while(True):
    
    date,hour,minute = get_time()
    # comp = input("Enter company name")
    comp = "RELIENCE"
    
    
    print(date,hour,minute, comp)

    # run query to diplay tweets
    try:
        display_tweets(date,hour,minute,comp)
    except:
        print("No tweets in this minute")
    
    # run query to diplay stocks
    display_stocks(date,hour,minute,comp)
    
    # sleep for around 1 minute
    # time.sleep(60)
    break

2022_06_05 22 17 RELIENCE
No tweets in this minute
/home/bdm/data_sources/stock_data_parquet/2022_06_05/stock_data_22_17.parquet
Empty DataFrame
Columns: [symbol, lastPrice, dayHigh, dayLow, closePrice, open]
Index: []


/home/bdm/.local/lib/python3.6/site-packages/pyspark/context.py:238: FutureWarning: Python 3.6 support is deprecated in Spark 3.2.
  FutureWarning


In [51]:
while(True):
    date,hour,min = get_time()
    date_ = os.path.join(path,date)
    parquet_name = 'stock_data'+'_'+str(hour)+'_'+str(min)+'.parquet'
    file_name = os.path.join(date_,parquet_name)
    print(file_name)
    df1 = spark.read.parquet(file_name)
    df2 = df1.select('symbol','lastPrice','dayHigh','dayLow','closePrice','open')
    # df2.show()
    pd_df = df2.toPandas()
    df_cmp = pd_df[pd_df['symbol']==company]
    display(HTML(df_cmp.to_html()))
    
    
    break
#/home/bdm/data_sources/stock_data_parquet/2022_06_05/stock_data2022_06_05_21_32.parquet

/home/bdm/data_sources/stock_data_parquet/2022_06_05/stock_data_22_18.parquet


,symbol,lastPrice,dayHigh,dayLow,closePrice,open
0,RELIANCE,2778.0,2817.35,2742.0,2779.5,2744.0


# USING WIDGETS IF POSSIBLE

In [33]:
display(HTML(df.to_html()))

,polarity,timestamp,sentiment,text,support_count
0,0.0,2022-06-03 16:26:59.547,Neutral,@drbillgibson Over reliance on antibiotics in these cases. Just encouraging resistance. Topical oestrogen is the way to go.,0


In [31]:
text = widgets.Text(description="Enter Company")
display(text)

# def process_results(sender):
#     comp = text.value
#     myquery = { "date": date , "companies":comp, "hour":hour, "minute":minute}
#     mydoc = tweetdb.find(myquery)
#     for x in mydoc:
#         df = pd.DataFrame(x['top_N_tweets'])
#         display(HTML(df.to_html()))


# text.on_submit(process_results)

Text(value='', description='Enter Company')

In [13]:
# # get todays date according to indian timezone
# # keeping buffer time to give time for processing (2 minutes)
# buffer_minutes = 2
# IN_time = pd.Timestamp.now('Asia/Kolkata')-pd.Timedelta(minutes=2)

# date = str(IN_time.date()).replace("-","_")
# date = "2022_06_03"

# hour = int(IN_time.hour)
# hour = 21

# minute = int(IN_time.minute)
# minute = 56



# # run query to extract result
# myquery = { "date": date , "companies":company, "hour":hour, "minute":minute}
# mydoc = tweetdb.find(myquery)
# for x in mydoc:
#     print("Extracted data")
#     df = pd.DataFrame(x['top_N_tweets'])
#     display(HTML(df.to_html()))